# Intake / Pangeo Catalog: Making It Easier To Consume Earth’s Climate and Weather Data

## Introduction

Computer simulations of the Earth’s climate and weather generate huge amounts of data. These data are often persisted on HPC systems or in the cloud across multiple data assets of a variety of formats (netCDF, zarr, etc...). Finding, investigating, loading these data assets into compute-ready data containers costs time and effort. The data user needs to know what data sets are available, the attributes describing each data set, before loading a specific data set and analyzing it. 


In this notebook, we demonstrate [intake-esm](https://github.com/intake/intake), a Python package and an [intake](https://github.com/intake/intake) plugin with an aim of facilitating
- the discovery of earth's climate and weather datasets.
- the ingestion of these datasets into xarray dataset containers.


The common/popular starting point for finding, investigating large datasets is with a data catalog. A *Data Catalog* is a collection of metadata, combined with search tools, that helps data analysts and other data users to find the data that they need. For a user to take full advantage of intake-esm, they need to point intake-esm to an ESM data catalog file. This is a JSON-based catalog file that conforms to the Earth System Model (ESM) collection specification. 

## ESM Collection Specification

The [ESM collection specification](https://github.com/NCAR/esm-collection-spec) provides a machine-readable (JSON) format for describing a wide range of climate and weather datasets. ESM’s goal is to make it easier to index and discover climate and weather data assets. An asset is any netCDF/HDF file or Zarr store that contains relevant data.


An ESM catalog serves as an inventory of available data, and provides information to explore the existing data assets. Additionaly, an ESM catalog can contain information about how to aggregate compatible groups of data assets into single xarray datasets. 


## Use Case: CMIP6 hosted on Google Cloud


The Coupled Model Intercomparison Project (CMIP) is an international collaborative effort to improve the knowledge about climate change and its impacts on the Earth System and on our society. [CMIP began in 1995](https://www.wcrp-climate.org/wgcm-cmip), and  today we are in its sixth phase (CMIP6). The CMIP6 data archive consists of data models created across approximately 30 working groups and 1,000 researchers investigating the urgent environmental problem of climate change. The CMIP6 will provide a wealth of information for the next Assessment Report (AR6) of the [Intergovernmental Panel on Climate Change](https://www.ipcc.ch/) (IPCC).

Last year, Pangeo partnered with Google Cloud to bring CMIP6 climate data to Google Cloud’s Public Datasets Program. You can read more about the dataset the process [here](https://cloud.google.com/blog/products/data-analytics/new-climate-model-data-now-google-public-datasets). 

For the rest of the notebook, we will demonstrate intake-esm's features using the intake-esm catalog for the CMIP6 data stored on Google Cloud Storage. This catalog resides [here](https://storage.googleapis.com/cmip6/pangeo-cmip6.json).


### Load an intake-esm catalog

In [1]:
# Import intake
import intake

In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json", sep=".")
col

,unique
activity_id,15
institution_id,33
source_id,70
experiment_id,102
member_id,140
table_id,29
variable_id,369
grid_label,10
zstore,267459
dcpp_init_year,60


The summary above is telling us that this catalog contains close to 268, 000 data assets. The first line (shown below) in the catalog contains the Ambient Aerosol Optical Thickness at 550nm (`variable_id='od550aer'`), as a function of latitude, longitude, time, and member_id, in an individual climate model experiment with the Taiwan Earth System Model 1.0 model (`source_id='TaiESM1'`) developed by the Taiwan Research Center for Environmental Changes (`instution_id='AS-RCEC'`). This model is **forced** by the experiment histSST (`experiment_id='histSST'`), which stands for *Historical transient with SSTs prescribed from historical*. This simulation was run as part of the AerChemMIP activity, which stands for *Aerosols and Chemistry Model Intercomparison Project*.

In [3]:
# List assets in the catalog
col.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,AerChemMIP,AS-RCEC,TaiESM1,histSST,r1i1p1f1,AERmon,od550aer,gn,gs://cmip6/AerChemMIP/AS-RCEC/TaiESM1/histSST/...,NaN
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
...,...,...,...,...,...,...,...,...,...,...
267454,ScenarioMIP,UA,MCM-UA-1-0,ssp585,r1i1p1f2,Omon,tos,gn,gs://cmip6/ScenarioMIP/UA/MCM-UA-1-0/ssp585/r1...,NaN
267455,ScenarioMIP,UA,MCM-UA-1-0,ssp585,r1i1p1f2,Omon,uo,gn,gs://cmip6/ScenarioMIP/UA/MCM-UA-1-0/ssp585/r1...,NaN
267456,ScenarioMIP,UA,MCM-UA-1-0,ssp585,r1i1p1f2,Omon,vo,gn,gs://cmip6/ScenarioMIP/UA/MCM-UA-1-0/ssp585/r1...,NaN
267457,ScenarioMIP,UA,MCM-UA-1-0,ssp585,r1i1p1f2,Omon,wo,gn,gs://cmip6/ScenarioMIP/UA/MCM-UA-1-0/ssp585/r1...,NaN


### Search 

We can now use intake-esm methods to search the collection. 

In [4]:
query = dict(
    experiment_id=['historical', 'ssp245', 'ssp585'],
    table_id='Amon',
    variable_id=['tas'],
    member_id = 'r1i1p1f1',
    grid_label='gr'
)

col_subset = col.search(require_all_on=["source_id"], **query)
col_subset.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id"]
].nunique()

,experiment_id,variable_id,table_id
source_id,,,
CIESM,3,1,1
EC-Earth3,3,1,1
EC-Earth3-Veg,3,1,1
FGOALS-f3-L,3,1,1
IPSL-CM6A-LR,3,1,1
KACE-1-0-G,3,1,1


In [5]:
# Find entries in the catalog
col_subset.keys()

['CMIP.CAS.FGOALS-f3-L.historical.Amon.gr',
 'CMIP.EC-Earth-Consortium.EC-Earth3.historical.Amon.gr',
 'CMIP.EC-Earth-Consortium.EC-Earth3-Veg.historical.Amon.gr',
 'CMIP.IPSL.IPSL-CM6A-LR.historical.Amon.gr',
 'CMIP.NIMS-KMA.KACE-1-0-G.historical.Amon.gr',
 'CMIP.THU.CIESM.historical.Amon.gr',
 'ScenarioMIP.CAS.FGOALS-f3-L.ssp245.Amon.gr',
 'ScenarioMIP.CAS.FGOALS-f3-L.ssp585.Amon.gr',
 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp245.Amon.gr',
 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.Amon.gr',
 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.Amon.gr',
 'ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp585.Amon.gr',
 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp245.Amon.gr',
 'ScenarioMIP.IPSL.IPSL-CM6A-LR.ssp585.Amon.gr',
 'ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp245.Amon.gr',
 'ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp585.Amon.gr',
 'ScenarioMIP.THU.CIESM.ssp245.Amon.gr',
 'ScenarioMIP.THU.CIESM.ssp585.Amon.gr']

In [6]:
dsets = col_subset.to_dataset_dict(zarr_kwargs={'consolidated': True})

Dataset(s):   0%|                                       | 0/18 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Dataset(s): 100%|██████████████████████████████| 18/18 [00:01<00:00, 11.05it/s]


In [7]:
dsets['ScenarioMIP.THU.CIESM.ssp585.Amon.gr']

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 192, lon: 288, member_id: 1, time: 1032)
Coordinates:
    height     float64 ...
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1032, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 303, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             ScenarioMIP
    branch_method:           standard
    branch_time_in_child:    735110.0
    branch_time_in_parent:   735110.0
    cmor_version:            3.5.0
    contact:                 yanluan@tsinghua.edu.cn
    creation_date:           2020-03-23T02:29:28Z
    data_specs_version:      01.00.29
    experiment:              update of RCP8.5 based on SSP5
    experiment_id:           ssp585
    external_variables:      areacella
    forcing_index:           1
    frequency:               mon
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.THU.CIESM.s...
    grid:                    gs1x1
    grid_label:              gr
    history:                 2020-03-23T02:29:28Z ;rewrote data to be consist...
    initialization_index:    1
    institution:             Department of Earth System Science, Tsinghua Uni...
    institution_id:          THU
    license:                 CMIP6 model data produced by Department of Earth...
    mip_era:                 CMIP6
    nominal_resolution:      100 km
    parent_activity_id:      CMIP
    parent_experiment_id:    historical
    parent_mip_era:          CMIP6
    parent_source_id:        CIESM
    parent_time_units:       days since 0001-01-01
    parent_variant_label:    r1i1p1f1
    physics_index:           1
    product:                 model-output
    realization_index:       1
    realm:                   atmos
    run_variant:             1rd realization
    source:                  CIESM (2019): \naerosol: prescribed MACv2-SP\nat...
    source_id:               CIESM
    source_type:             AOGCM
    status:                  2020-05-01;created; by gcs.cmip6.ldeo@gmail.com
    sub_experiment:          none
    sub_experiment_id:       none
    table_id:                Amon
    table_info:              Creation Date:(20 February 2019) MD5:510997cd0a2...
    title:                   CIESM output prepared for CMIP6
    tracking_id:             hdl:21.14100/eba65a5e-f036-487a-a96e-757e2293ed66
    variable_id:             tas
    variant_label:           r1i1p1f1
    intake_esm_varname:      tas
    intake_esm_dataset_key:  ScenarioMIP.THU.CIESM.ssp585.Amon.gr